In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from selenium.webdriver.support.ui import Select
import re
from selenium.webdriver.chrome.options import Options
import os
import zipfile
import shutil
from Bio.Seq import Seq
import pandas as pd
import pickle

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome("C://project_IP//chromedriver.exe",chrome_options=chrome_options)


C:\Users\EHOUT\AppData\Local\Temp/ipykernel_19156/2679306720.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("C://project_IP//chromedriver.exe",chrome_options=chrome_options)


In [3]:
def exp_to_short(amino_acid):
    ''' 
      A function which can convert expanded forms
      like Ala to A
    '''

    dicts={'Ala':'a', 'Asx':'b', 'Cys':'c',
           'Asp':'d', 'Glu':'e', 'Phe':'f',
           'Gly':'g', 'His':'h', 'Ile':'i',
           'Lys':'k', 'Leu':'l', 'Met':'m',
           'Asn':'n', 'Pro':'p', 'Gln':'q',
           'Arg':'r', 'Ser':'s', 'Thr':'t',
           'Sec':'u', 'Val':'v', 'Trp':'w',
           'Xaa':'x', 'Tyr':'y', 'Glx':'z'}
    
    return dicts[amino_acid]



def translate(sequence):
    coding_dna = Seq(sequence)
    prt=str(coding_dna.translate()) #translate dna to protein
    return prt

In [4]:
# get sequences from sequence listing text file
def get_data(file, upper=True):
    file_content=file
    
        
    seq_no=[]
    seq_len=[]
    seq_type=[]
    organism=[]
    other_info=[]
    trnsl_seq=[]
    
    chk="<400>" #to check the previous label
    for i in range(len(file_content)):
        x=file_content[i].split()
        
        #get sequence number
        if(len(x)>0 and x[0]=='<210>' and chk=='<400>'):
            seq_no.append(file_content[i][6:len(file_content[i])])
            chk=x[0]
            
        #get sequence length
        if(len(x)>0 and x[0]=='<211>' and chk=='<210>'):
            seq_len.append(file_content[i][6:len(file_content[i])])
            chk=x[0]
        
        #get sequence type
        if(len(x)>0 and x[0]=='<212>' and chk=='<211>'):
            seq_type.append(file_content[i][6:len(file_content[i])])
            chk=x[0]
            
            
        #get organism name
        if(len(x)>0 and x[0]=='<213>' and chk=='<212>'):
            organism.append(file_content[i][6:len(file_content[i])])
            chk=x[0]
            
            
        #get other information
        if(len(x)>0 and x[0]=='<223>' and chk=='<213>'):
            other_info.append(file_content[i][6:len(file_content[i])])
            chk=x[0]
            
        # if any attribute is missing add none to the list
        if(len(x)>0 and x[0]=='<400>'):
            if(len(other_info)!=len(seq_no)):
                #print(len(seq_no), len(other_info))
                
                other_info.append(" ")
            
            if(len(seq_len)!=len(seq_no)):
                seq_len.append(0)
                
            if(len(seq_type)!=len(seq_no)):
                seq_type.append("None")
                
            if(len(organism)!=len(seq_no)):
                organism.append("None")
                
            chk=x[0]
            
    
    seq_code=[] #getting the sequence code    
    i=0    
    while(i<len(file_content)):
        x=file_content[i].split()
        if(len(x)>0 and x[0]=='<400>'):
            i+=1
            
            temp=""
            
            while(x[0]!='<210>' and i<len(file_content)): #append all sequence between 400 and 210
                x=file_content[i].split()
                if(len(x)>0):
                    for j in range(len(x)):
                        #convert expanded forms like 'Ala' to 'a'
                        try:    
                            if(upper==True):
                                temp+=exp_to_short(x[j]).upper()
                            else:
                                temp+=exp_to_short(x[j])
                        except:
                            # don't add the digits in sequence 
                            if(x[j].isdigit()==False and x[0]!='<210>'):
                               temp+=x[j]
                else:
                    x.append('0') #to nullify the exception
                i+=1
                
            sz=len(seq_code)
            if(seq_type[sz][-3:]!="PRT"): #if sequence type is not protein
                prt=translate(temp) 
                trnsl_seq.append(prt)
            else:
                trnsl_seq.append(temp)
                
            seq_code.append(temp) 
            
        else:
            i+=1
               
             
    df_dict={'Sequence_ID_number': seq_no, 'Sequence_length':seq_len,
             'Sequence_type':seq_type, 'Organism':organism,
             'Other_info': other_info, 'Sequence':seq_code,
             'Protein': trnsl_seq}
    
    
    df=pd.DataFrame(df_dict)
    
    return df

In [5]:
# get sequences from wipo database
def get_seq_wipo(pat_no):
    url="https://patentscope.wipo.int/search/en/search.jsf"
    driver.get(url)
    
    time.sleep(5)
    
    search_box=driver.find_element_by_id('simpleSearchForm:fpSearch:input')
    search_box.send_keys(pat_no)
    
    #WO2018081194
    search_box.send_keys(Keys.ENTER)
    
    time.sleep(5)
    
    # find the column having documents
    m=0
    ts=driver.find_elements_by_xpath('//*[@id="detailMainForm:MyTabViewId"]/ul/li')
    for i in range(len(ts)):
        if(ts[i].text=='Documents'):
            m=i+1
            
    try:
        #select the documents
        documents=driver.find_element_by_xpath('//*[@id="detailMainForm:MyTabViewId"]/ul/li['+str(m)+']/a')
        documents.send_keys(Keys.ENTER)
    except Exception:
        pass
    
    time.sleep(5)
    
    x=driver.find_elements_by_class_name('ui-datatable')
    iD=x[1].get_attribute('id') #get the second table
    
    p='//*[@id="'+iD+'"]/div[2]/table/tbody/tr' #rows of second table
    table=driver.find_elements_by_xpath(p)
    
    #find the row in which sequence listing text file is present
    fnd=0
    for i in range(len(table)):
        s=table[i].text
        if(s.find('Sequence Listing')!=-1 and s.find('TXT')!=-1):
            fnd=i+1
            break
    
    time.sleep(5)
    
    #download the text file
    sind=driver.find_element_by_xpath('//*[@id="'+iD+'"]/div[2]/table/tbody/'+'/tr['+str(fnd)+']/td[3]/div/span[1]/a')
    sind.send_keys(Keys.ENTER)
    
    tesxt=driver.find_element_by_tag_name('body')
    file=tesxt.text.split('\n')
    df=get_data(file)
    
    return df



In [6]:
dicts={'Ala':'a', 'Asx':'b', 'Cys':'c',
       'Asp':'d', 'Glu':'e', 'Phe':'f',
       'Gly':'g', 'His':'h', 'Ile':'i',
       'Lys':'k', 'Leu':'l', 'Met':'m',
       'Asn':'n', 'Pro':'p', 'Gln':'q',
       'Arg':'r', 'Ser':'s', 'Thr':'t',
       'Sec':'u', 'Val':'v', 'Trp':'w',
       'Xaa':'x', 'Tyr':'y', 'Glx':'z'}

lis_amin=[]
for key in dicts:
    lis_amin.append(key)
    
lis_chk=['a', 't', 'g', 'c']

In [7]:
# Get sequences for us patents from website "https://appft1.uspto.gov/netahtml/PTO/srchnum.html"
def get_us_patents_1(patent_no):
    url="https://appft1.uspto.gov/netahtml/PTO/srchnum.html"
    driver.get(url)

    search_box=driver.find_element_by_xpath('//*[@id="qry"]')
    search_box.send_keys(patent_no)
    #20090023654
    search_box.send_keys(Keys.ENTER)

    lnk=driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[3]/a')
    driver.get(lnk.get_attribute('href'))

    el = driver.find_elements_by_tag_name('p')
    ans=-1
    for i in range(len(el)):
        tsxt=el[i].text
        if(tsxt.find('Sequence CWU')!=-1):
            ans=i
            break

    import re
    data=el[ans].text    


    name=[]
    new_data=[]
    res = re.split('DNA|PRT', data)
    for i in range(1,len(res)):
        for j in range(len(res[i])-2):
            if(res[i][j].isdigit()==True and res[i][j+1].isalpha()==True): #pattern number followed by alphabet
                
                if(res[i][j+1].isupper()):
                    cnt=0
                    if res[i][j+1:j+4] in lis_amin: # Check if three letter word is amino acid from dicts
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
                    
                else:
                    cnt=0
                    for k in range(j+1,j+10):
                        if(res[i][k]!=' '):
                            if res[i][k] not in lis_chk: #check if 10 letter word contains 'a','t','g','c'
                                cnt+=1
                                break
                    if(cnt==0):
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
            elif(res[i][j].isdigit()==True and res[i][j+2].isalpha()==True): #if the pattern is changed number followed by space then alphabet
                
                
                if(res[i][j+2].isupper()):
                    cnt=0
                    if res[i][j+2:j+5] in lis_amin:
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
                    
                else:
                    cnt=0
                    for k in range(j+2,j+10):
                        if(res[i][k]!=' '):
                            if res[i][k] not in lis_chk:
                                cnt+=1
                                break
                    if(cnt==0):
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
                                
                

    fin_data=[]
    for i in range(len(new_data)):
        fin_data.append(re.sub(r'[^a-zA-Z ]+', '', new_data[i])) #clean the data

    seq=[]
    for i in range(len(fin_data)):
        ix=fin_data[i].find("SEQ ID NO") #remove everything after 'SEQ ID NO' if present
        if(ix!=-1):
            fin_data[i]=fin_data[i][:ix]
        x=fin_data[i].split()
        temp=""
        if(fin_data[i].islower()):
            for j in range(len(x)):
                temp+=x[j]
        else:
            for j in range(len(x)):
                try:    
                    
                    temp+=exp_to_short(x[j]).upper()
                    
                except:
                      pass
                  
        seq.append(temp)
        
    dicts={"Name": name, "Sequence": seq}
    df=pd.DataFrame(dicts)
    return df
    
   


In [8]:
# Get Us patents from the website "https://patft.uspto.gov/netahtml/PTO/srchnum.htm"
def get_us_patents_2(patent_no):
    url="https://patft.uspto.gov/netahtml/PTO/srchnum.htm"
    driver.get(url)

    time.sleep(3)
    search_box=driver.find_element_by_xpath('//*[@id="qry"]')
    search_box.send_keys(patent_no)
    #20090023654
    search_box.send_keys(Keys.ENTER)

    time.sleep(3)
    el = driver.find_element_by_tag_name('body')
    el2=el.text
    ans=el2.find('SEQUENCE LISTINGS')
    '''for i in range(len(el2)):
        if(el2[i]=='SEQUENCE LISTINGS'):
            ans=i+1
            break
        
    for i in range(ans,len(el2)):
        if(len(el2[i])>10):
            ans=i
            break'''


    data=el2[ans:] 


    name=[]
    new_data=[]
    res = re.split('DNA|PRT', data)
    for i in range(1,len(res)):
        for j in range(len(res[i])-2):
            if(res[i][j].isdigit()==True and res[i][j+1].isalpha()==True):
                
                if(res[i][j+1].isupper()):
                    cnt=0
                    if res[i][j+1:j+4] in lis_amin:
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
                    
                else:
                    cnt=0
                    for k in range(j+1,j+10):
                        if(res[i][k]!=' '):
                            if res[i][k] not in lis_chk:
                                cnt+=1
                                break
                    if(cnt==0):
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
            elif(res[i][j].isdigit()==True and res[i][j+2].isalpha()==True):
                
                
                if(res[i][j+2].isupper()):
                    cnt=0
                    if res[i][j+2:j+5] in lis_amin:
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
                    
                else:
                    cnt=0
                    for k in range(j+2,j+10):
                        if(res[i][k]!=' '):
                            if res[i][k] not in lis_chk:
                                cnt+=1
                                break
                    if(cnt==0):
                        name.append(res[i][:j])
                        new_data.append(res[i][j:])
                        break
                                
                

    fin_data=[]
    for i in range(len(new_data)):
        fin_data.append(re.sub(r'[^a-zA-Z ]+', '', new_data[i]))

    seq=[]
    for i in range(len(fin_data)):
        #ix=fin_data[i].find("SEQ ID NO")
        #fin_data[i]=fin_data[i][:ix]
        x=fin_data[i].split()
        temp=""
        if(fin_data[i].islower()):
            for j in range(len(x)):
                temp+=x[j]
        else:
            for j in range(len(x)):
                try:    
                    
                    temp+=exp_to_short(x[j]).upper()
                    
                except:
                      pass
                  
        seq.append(temp)
        
    dicts={"Name": name, "Sequence": seq}
    df=pd.DataFrame(dicts)
    return df
    

In [9]:
t=get_seq_wipo('WO2018081194')

In [10]:
t


,Sequence_ID_number,Sequence_length,Sequence_type,Organism,Other_info,Sequence,Protein
0,1,1578,DNA,Holdemania massiliensis,,atggaacagatttacaatggaatgctgccgcttggtcttagccaac...,MEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLLAEQSPTIIYDK...
1,2,1578,DNA,Artificial Sequence,derived from Holdemania massiliensis sequence,atggaacagatttataatggcatgttaccgctgggtttgtcccagc...,MEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLLAEQSPTIIYDK...
2,3,1578,DNA,Artificial Sequence,derived from Holdemania massiliensis,atggagcagatctacaatgggatgctgccactcggcctgagccagc...,MEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLLAEQSPTIIYDK...
3,4,525,PRT,Holdemania massiliensis,,MEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLLAEQSPTIIYDK...,MEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLLAEQSPTIIYDK...
4,5,13,PRT,Holdemania massiliensis,,YGTHLLLDIRLGG,YGTHLLLDIRLGG
5,6,1611,DNA,Holdemania massiliensis,,atgaggctaaggagtacgaaaggagaaatcaaaatggaacagattt...,MRLRSTKGEIKMEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLL...
6,7,1611,DNA,Holdemania massiliensis,,atgcgtcttcgttctactaaaggcgaaatcaagatggaacagattt...,MRLRSTKGEIKMEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLL...
7,8,1611,DNA,Holdemania massiliensis,,atgcgccttcggagcacaaaaggagaaatcaagatggagcagatct...,MRLRSTKGEIKMEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLL...
8,9,536,PRT,Holdemania massiliensis,,MRLRSTKGEIKMEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLL...,MRLRSTKGEIKMEQIYNGMLPLGLSQRLNEPVMMLEKGEQAKPQLL...


In [12]:
u=get_us_patents_1('20110154524')

In [13]:
u

,Name,Sequence
0,Bacillus thuringiensisPEPTIDE(0)...(0)Cry1F,MENNIQNQCVPYNCLNNPEVEILNEERSTGRLPLDISLSLTRFLLS...
1,Bacillus thuringiensisPEPTIDE(0)...(0)Cry34Ab1,MSAREVHIDVNNKTGHTLQLEDKTKLDGGRWRTSPTNVANDQIKTF...
2,Bacillus thuringiensisPEPTIDE(0)...(0)Cry35Ab1,MLDTNKVYEISNHANGLYAATYLSLDDSGVSLMNKNDDDIDDYNLK...
3,Streptomyces viridochromogenesPEPTIDE(0)...(0)...,MSPERRPVEIRPATAADMAAVCDIVNHYIETSTVNFRTEPQTPQEW...
4,region of Plasmid PHP27118,gtttacccgccaatatatcctgtcaaacactgatagtttaaacgct...
5,Artificial SequenceThe complete sequence of th...,cgatctctgcagacgtttgaacaatggaagaaacacacctccacgt...
6,Artificial Sequencemisc_feature(0)...(0)Primer...,agggcctcatatccggtctcagttac
7,Artificial Sequencemisc_feature(0)...(0)Primer...,cacgctgcactgcaggcatgcaagctggc
8,Artificial Sequencemisc_feature(0)...(0)Primer...,ccgctgtatcacaagggctggtacc
9,Artificial Sequencemisc_feature(0)...(0)Primer...,ggagcccgtgtagagcatgacgatc


In [16]:
u2=get_us_patents_2('9382550')

In [17]:
u2

,Name,Sequence
0,Artificial SequenceNucleotide sequence compris...,cttttatcatataagtgcctatataaataaagtttagtcagtcaaa...
1,Artificial SequenceNucleotide sequence compris...,atagcgcgcaaactaggataaattatcgcgcgcggtgtcatctatc...
2,Artificial SequenceOligonucleotide primer GHI057,atagcgcgcaaactagga
3,Artificial SequenceOligonucleotide primer MAE115,tcggcaatgttatcggcatc
4,Artificial SequenceOligonucleotide primer HVH022,ccactgcttggcatttgag
5,Artificial SequenceOligonucleotide primer GHI065,gggccggataaaattagcct
6,Artificial SequenceOligonucleotide primer HVH024,gtcattgtagggagtttgtc
7,Artificial SequenceOligonucleotide primer DPA312,tgcctcttgaactgtagc
8,Artificial SequenceOligonucleotide primer GHI066,agataaaatcgtcagtgctg
9,Artificial SequenceOligonucleotide primer GHI047,tgcctcttgaactgtagc


Get all the sequences for all the patents from the sheet

In [ ]:
data_file=pd.read_excel("patent_data.xlsx")
df1=pd.DataFrame()
df2=pd.DataFrame()
dint_wrk=[]
wrk=[]

for i in range(data_file.shape[0]):
    
    
    if(data_file['Publication_No'][i][0]=='W'):
        
        try:
            '''if(loaded_list[i][2:].lower().islower()):
                dwnload_sequence(loaded_list[i][:-2])
            else:
                dwnload_sequence(loaded_list[i])'''
            
            if(data_file['Publication_No'][i][2:].lower().islower()):
                df_data=get_seq_wipo(data_file['Publication_No'][i][:-2])
                pat_nam=[data_file['Publication_No'][i]]*df_data.shape[0]
                df_data['patent_number']=pat_nam
                df1=pd.concat((df1,df_data))
            
            else:
                df_data=get_seq_wipo(data_file['Publication_No'][i])
                pat_nam=[data_file['Publication_No'][i]]*df_data.shape[0]
                df_data['patent_number']=pat_nam
                df1=pd.concat((df1,df_data))
            
            
            wrk.append(data_file['Publication_No'][i])
            if(i%200==0):
                df1.to_csv("fetched_wipo"+str(i)+".csv")
                df2.to_csv("fetched_uspto"+str(i)+".csv")
                dn=0
                
                open_file = open("worked"+str(i), "wb")
                pickle.dump(wrk, open_file)
                open_file.close()
                
                
                open_file = open("dintworked"+str(i), "wb")
                pickle.dump(dint_wrk, open_file)
                open_file.close()
        
        
        
        
        
        except:
            dint_wrk.append(data_file['Publication_No'][i])
    
    else:
        try:
            if(data_file['Publication_No'][i][2:].lower().islower()):
                df_data=get_us_patents_1(data_file['Publication_No'][i][2:-2])
                pat_nam=[data_file['Publication_No'][i]]*df_data.shape[0]
                df_data['patent_number']=pat_nam
                df2=pd.concat((df2,df_data))
            else:
                df_data=get_us_patents_1(data_file['Publication_No'][i][2:])
                pat_nam=[data_file['Publication_No'][i]]*df_data.shape[0]
                df_data['patent_number']=pat_nam
                df2=pd.concat((df2,df_data))
            
            
            wrk.append(data_file['Publication_No'][i])
            if(i%200==0):
                df1.to_csv("fetched_wipo"+str(i)+".csv")
                df2.to_csv("fetched_uspto"+str(i)+".csv")
                dn=0
                
                open_file = open("worked"+str(i), "wb")
                pickle.dump(wrk, open_file)
                open_file.close()
                
                
                open_file = open("dintworked"+str(i), "wb")
                pickle.dump(dint_wrk, open_file)
                open_file.close()
        
        except:
            try:
                if(data_file['Publication_No'][i][2:].lower().islower()):
                    df_data=get_us_patents_2(data_file['Publication_No'][i][2:-2])
                    pat_nam=[data_file['Publication_No'][i]]*df_data.shape[0]
                    df_data['patent_number']=pat_nam
                    df2=pd.concat((df2,df_data))
                else:
                    df_data=get_us_patents_2(data_file['Publication_No'][i][2:])
                    pat_nam=[data_file['Publication_No'][i]]*df_data.shape[0]
                    df_data['patent_number']=pat_nam
                    df2=pd.concat((df2,df_data))
                    
                wrk.append(data_file['Publication_No'][i])
                if(i%200==0):
                    df1.to_csv("fetched_wipo"+str(i)+".csv")
                    df2.to_csv("fetched_uspto"+str(i)+".csv")
                    dn=0
                    
                    open_file = open("worked"+str(i), "wb")
                    pickle.dump(wrk, open_file)
                    open_file.close()
                    
                    
                    open_file = open("dintworked"+str(i), "wb")
                    pickle.dump(dint_wrk, open_file)
                    open_file.close()
            
            except:
                dint_wrk.append(data_file['Publication_No'][i])
                
open_file = open("worked"+str(i), "wb")
pickle.dump(wrk, open_file)
open_file.close()


open_file = open("dintworked"+str(i), "wb")
pickle.dump(dint_wrk, open_file)
open_file.close()
                
df1.to_csv('fetched_wipo_all.csv')
df2.to_csv('fetched_uspto_all.csv')